# import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint
import time
import sys
import platform

In [2]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.aer import noise
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

# Environment

In [3]:
# device version
pprint(platform.machine())
pprint(platform.version())
pprint(platform.platform())
pprint(platform.uname())
pprint(platform.system())
pprint(platform.processor())

'x86_64'
('Darwin Kernel Version 19.5.0: Tue May 26 20:41:44 PDT 2020; '
 'root:xnu-6153.121.2~2/RELEASE_X86_64')
'Darwin-19.5.0-x86_64-i386-64bit'
uname_result(system='Darwin', node='Yang-no-MBP', release='19.5.0', version='Darwin Kernel Version 19.5.0: Tue May 26 20:41:44 PDT 2020; root:xnu-6153.121.2~2/RELEASE_X86_64', machine='x86_64', processor='i386')
'Darwin'
'i386'


In [4]:
# python version
pprint(sys.version)

('3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 15:01:53) \n'
 '[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]')


In [5]:
# qiskit version
qiskit.__qiskit_version__

{'qiskit-terra': '0.16.1',
 'qiskit-aer': '0.7.2',
 'qiskit-ignis': '0.6.0.dev0+6405d16',
 'qiskit-ibmq-provider': '0.12.0.dev0+14b4ce6',
 'qiskit-aqua': None,
 'qiskit': None}

# Measure the time of tensored calibration for GHZ state from size 2 to 12
- Testing for "pseudo_inverse" method
- Using the original TensoredFilter.apply() function

In [6]:
import pdb

In [14]:
n = 5
print("experiment of ", n, "-qubit GHZ state")

# create n qubit tensored measurement calibration circuits
qr = qiskit.QuantumRegister(n)
# mit_pattern = [[i] for i in range(n)]
mit_pattern = [[0, 2], [4], [1, 3]] # , [2]]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
print("mit pattern: ", mit_pattern)
print("number of calibration circuits: ", len(meas_calibs))

# prepare noise model of n qubits
noise_model = noise.NoiseModel()
for qi in range(n):
    if qi <= 1:
        continue
    read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.25,0.75]])
    noise_model.add_readout_error(read_err, [qi])

# execute the calibration circuits
job = qiskit.execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=5000, noise_model=noise_model)
cal_results = job.result()
meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
print(meas_fitter.cal_matrices)

# create n-qubit GHZ state
m = n
cr = ClassicalRegister(m)
ghz = QuantumCircuit(qr, cr)
ghz.h(qr[0])
for i in range(1,m):
    ghz.cx(qr[0], qr[i])
ghz.barrier()
for i in range(m):
    ghz.measure(qr[i],cr[i])
# ghz.draw("mpl")

# execute GHZ circuit
job = qiskit.execute([ghz], backend=Aer.get_backend("qasm_simulator"), shots=5000, noise_model=noise_model)
noisy_counts = job.result().get_counts()

# Get the filter object
meas_filter = meas_fitter.filter

t1 = time.time()
# Results with mitigation
# pdb.set_trace()
mitigated_counts = meas_filter.apply(noisy_counts, "pseudo_inverse", meas_layout=list(range(n)))
t2 = time.time()

print("time of mitigation for", n, "qubits GHZ state: ", t2 - t1, "[s]")

pprint(noisy_counts)
pprint(mitigated_counts)

print()

experiment of  5 -qubit GHZ state
mit pattern:  [[0, 2], [4], [1, 3]]
number of calibration circuits:  4
[array([[0.9026, 0.    , 0.246 , 0.    ],
       [0.    , 0.8954, 0.    , 0.2534],
       [0.0974, 0.    , 0.754 , 0.    ],
       [0.    , 0.1046, 0.    , 0.7466]]), array([[0.8962, 0.2469],
       [0.1038, 0.7531]]), array([[0.903 , 0.    , 0.254 , 0.    ],
       [0.    , 0.9024, 0.    , 0.2582],
       [0.097 , 0.    , 0.746 , 0.    ],
       [0.    , 0.0976, 0.    , 0.7418]])]
self._mit_pattern
[[0, 2], [4], [1, 3]]
time of mitigation for 5 qubits GHZ state:  0.0022819042205810547 [s]
{'00000': 1902,
 '00011': 30,
 '00100': 194,
 '00111': 113,
 '01000': 201,
 '01011': 104,
 '01100': 21,
 '01111': 360,
 '10000': 197,
 '10011': 128,
 '10100': 21,
 '10111': 336,
 '11000': 31,
 '11011': 338,
 '11100': 3,
 '11111': 1021}
{'00000': 2811.0252402470087,
 '00001': -355.91893944707186,
 '00010': -358.5394197432016,
 '00011': 20.70818439837013,
 '00100': 290.62013958884035,
 '00101': -42.

In [13]:
n = 5
print("experiment of ", n, "-qubit GHZ state")

# create n qubit tensored measurement calibration circuits
qr = qiskit.QuantumRegister(n)
# mit_pattern = [[i] for i in range(n)]
mit_pattern = [[0, 2], [1], [4, 3]] # , [2]]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
print("mit pattern: ", mit_pattern)
print("number of calibration circuits: ", len(meas_calibs))

# prepare noise model of n qubits
noise_model = noise.NoiseModel()
for qi in range(n):
    if qi != 3:
        continue
    read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.25,0.75]])
    noise_model.add_readout_error(read_err, [qi])

# execute the calibration circuits
job = qiskit.execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=5000, noise_model=noise_model)
cal_results = job.result()
meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
print(meas_fitter.cal_matrices)

# create n-qubit GHZ state
m = n
cr = ClassicalRegister(m)
ghz = QuantumCircuit(qr, cr)
ghz.h(qr[0])
for i in range(1,m):
    ghz.cx(qr[0], qr[i])
ghz.barrier()
for i in range(m):
    ghz.measure(qr[i],cr[i])
# ghz.draw("mpl")

# execute GHZ circuit
job = qiskit.execute([ghz], backend=Aer.get_backend("qasm_simulator"), shots=5000, noise_model=noise_model)
noisy_counts = job.result().get_counts()

# Get the filter object
meas_filter = meas_fitter.filter

t1 = time.time()
# Results with mitigation
# pdb.set_trace()
mitigated_counts = meas_filter.apply(noisy_counts, "pseudo_inverse", meas_layout=list(range(n)))
t2 = time.time()

print("time of mitigation for", n, "qubits GHZ state: ", t2 - t1, "[s]")

pprint(noisy_counts)
pprint(mitigated_counts)

print()

experiment of  5 -qubit GHZ state
mit pattern:  [[0, 2], [1], [4, 3]]
number of calibration circuits:  4
[array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[1., 0.],
       [0., 1.]]), array([[0.8952, 0.    , 0.2522, 0.    ],
       [0.    , 0.9002, 0.    , 0.2504],
       [0.1048, 0.    , 0.7478, 0.    ],
       [0.    , 0.0998, 0.    , 0.7496]])]
self._mit_pattern
[[0, 2], [1], [4, 3]]
time of mitigation for 5 qubits GHZ state:  0.002147674560546875 [s]
{'00000': 2342, '01000': 248, '10111': 595, '11111': 1815}
{'00000': 2723.713219284604,
 '00111': -233.37325038880238,
 '01000': 286.08925823330253,
 '01111': -699.4090489381344,
 '10000': -381.71321928460316,
 '10111': 828.3732503888021,
 '11000': -38.08925823330253,
 '11111': 2514.409048938134}



In [12]:
n = 5
print("experiment of ", n, "-qubit GHZ state")

# create n qubit tensored measurement calibration circuits
qr = qiskit.QuantumRegister(n)
# mit_pattern = [[i] for i in range(n)]
mit_pattern = [[0, 2], [1], [3, 4]] # , [2]]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
print("mit pattern: ", mit_pattern)
print("number of calibration circuits: ", len(meas_calibs))

# prepare noise model of n qubits
noise_model = noise.NoiseModel()
for qi in range(n):
    if qi != 3:
        continue
    read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.25,0.75]])
    noise_model.add_readout_error(read_err, [qi])

# execute the calibration circuits
job = qiskit.execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=5000, noise_model=noise_model)
cal_results = job.result()
meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
print(meas_fitter.cal_matrices)

# create n-qubit GHZ state
m = n
cr = ClassicalRegister(m)
ghz = QuantumCircuit(qr, cr)
ghz.h(qr[0])
for i in range(1,m):
    ghz.cx(qr[0], qr[i])
ghz.barrier()
for i in range(m):
    ghz.measure(qr[i],cr[i])
# ghz.draw("mpl")

# execute GHZ circuit
job = qiskit.execute([ghz], backend=Aer.get_backend("qasm_simulator"), shots=5000, noise_model=noise_model)
noisy_counts = job.result().get_counts()

# Get the filter object
meas_filter = meas_fitter.filter

t1 = time.time()
# Results with mitigation
# pdb.set_trace()
mitigated_counts = meas_filter.apply(noisy_counts, "pseudo_inverse", meas_layout=list(range(n)))
t2 = time.time()

print("time of mitigation for", n, "qubits GHZ state: ", t2 - t1, "[s]")

pprint(noisy_counts)
pprint(mitigated_counts)

print()

experiment of  5 -qubit GHZ state
mit pattern:  [[0, 2], [1], [3, 4]]
number of calibration circuits:  4
[array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[1., 0.],
       [0., 1.]]), array([[0.9054, 0.253 , 0.    , 0.    ],
       [0.0946, 0.747 , 0.    , 0.    ],
       [0.    , 0.    , 0.905 , 0.2442],
       [0.    , 0.    , 0.095 , 0.7558]])]
self._mit_pattern
[[0, 2], [1], [3, 4]]
time of mitigation for 5 qubits GHZ state:  0.0025038719177246094 [s]
{'00000': 2240, '01000': 249, '10111': 623, '11111': 1888}
{'00000': 2564.8068669527897,
 '00111': -241.59871244635198,
 '01000': 284.79751815980615,
 '01111': -697.7142857142857,
 '10000': -324.80686695278933,
 '10111': 864.598712446352,
 '11000': -35.7975181598063,
 '11111': 2585.714285714285}



In [11]:
n = 5
print("experiment of ", n, "-qubit GHZ state")

# create n qubit tensored measurement calibration circuits
qr = qiskit.QuantumRegister(n)
# mit_pattern = [[i] for i in range(n)]
mit_pattern = [[0, 2], [1], [3, 4]] # , [2]]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
print("mit pattern: ", mit_pattern)
print("number of calibration circuits: ", len(meas_calibs))

# prepare noise model of n qubits
noise_model = noise.NoiseModel()
for qi in range(n):
    if qi != 1:
        continue
    read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.25,0.75]])
    noise_model.add_readout_error(read_err, [qi])

# execute the calibration circuits
job = qiskit.execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=5000, noise_model=noise_model)
cal_results = job.result()
meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
print(meas_fitter.cal_matrices)

# create n-qubit GHZ state
m = n
cr = ClassicalRegister(m)
ghz = QuantumCircuit(qr, cr)
ghz.h(qr[0])
for i in range(1,m):
    ghz.cx(qr[0], qr[i])
ghz.barrier()
for i in range(m):
    ghz.measure(qr[i],cr[i])
# ghz.draw("mpl")

# execute GHZ circuit
job = qiskit.execute([ghz], backend=Aer.get_backend("qasm_simulator"), shots=5000, noise_model=noise_model)
noisy_counts = job.result().get_counts()

# Get the filter object
meas_filter = meas_fitter.filter

t1 = time.time()
# Results with mitigation
# pdb.set_trace()
mitigated_counts = meas_filter.apply(noisy_counts, "pseudo_inverse", meas_layout=list(range(n)))
t2 = time.time()

print("time of mitigation for", n, "qubits GHZ state: ", t2 - t1, "[s]")

pprint(noisy_counts)
pprint(mitigated_counts)

print()

experiment of  5 -qubit GHZ state
mit pattern:  [[0, 2], [1], [3, 4]]
number of calibration circuits:  4
[array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[0.9031, 0.2498],
       [0.0969, 0.7502]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])]
self._mit_pattern
[[0, 2], [1], [3, 4]]
time of mitigation for 5 qubits GHZ state:  0.0021009445190429688 [s]
{'00000': 2243, '00010': 270, '11101': 666, '11111': 1821}
{'00000': 2472.451553650697,
 '00010': 40.548446349303504,
 '11101': 68.49441298025408,
 '11111': 2418.505587019747}



In [16]:
n = 5
print("experiment of ", n, "-qubit GHZ state")

# create n qubit tensored measurement calibration circuits
qr = qiskit.QuantumRegister(n)
# mit_pattern = [[i] for i in range(n)]
mit_pattern = [[0, 2], [1], [3, 4]] # , [2]]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
print("mit pattern: ", mit_pattern)
print("number of calibration circuits: ", len(meas_calibs))

# prepare noise model of n qubits
noise_model = noise.NoiseModel()
for qi in range(n):
    if qi != 1:
        continue
    read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.25,0.75]])
    noise_model.add_readout_error(read_err, [qi])

# execute the calibration circuits
job = qiskit.execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=5000, noise_model=noise_model)
cal_results = job.result()
meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
print(meas_fitter.cal_matrices)

# create n-qubit GHZ state
m = n
cr = ClassicalRegister(m)
ghz = QuantumCircuit(qr, cr)
ghz.h(qr[0])
for i in range(1,m):
    ghz.cx(qr[0], qr[i])
ghz.barrier()
for i in range(m):
    ghz.measure(qr[i],cr[i])
# ghz.draw("mpl")

# execute GHZ circuit
job = qiskit.execute([ghz], backend=Aer.get_backend("qasm_simulator"), shots=5000, noise_model=noise_model)
noisy_counts = job.result().get_counts()

# Get the filter object
meas_filter = meas_fitter.filter

t1 = time.time()
# Results with mitigation
# pdb.set_trace()
mitigated_counts = meas_filter.apply(noisy_counts, "least_squares", meas_layout=list(range(n)))
t2 = time.time()

print("time of mitigation for", n, "qubits GHZ state: ", t2 - t1, "[s]")

pprint(noisy_counts)
pprint(mitigated_counts)

print()

experiment of  5 -qubit GHZ state
mit pattern:  [[0, 2], [1], [3, 4]]
number of calibration circuits:  4
[array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[0.9031, 0.2492],
       [0.0969, 0.7508]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])]
self._mit_pattern
[[0, 2], [1], [3, 4]]
time of mitigation for 5 qubits GHZ state:  0.047551870346069336 [s]
{'00000': 2269, '00010': 253, '11101': 612, '11111': 1866}
{'00000': 156.23143653981128,
 '00001': 156.26275483344267,
 '00010': 156.27565208845417,
 '00011': 156.22449383028948,
 '00100': 156.24621150395058,
 '00101': 156.24936041284778,
 '00110': 156.2685238564137,
 '00111': 156.22273866470704,
 '01000': 156.22026087431297,
 '01001': 156.23988183640628,
 '01010': 156.24398373139695,
 '01011': 156.27306557324735,
 '01100': 156.219492108698,
 '01101': 156.27578326919402,
 '01110': 156.23505614560287,
 '01111': 156.270602219